In [12]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
ct_chemberta2_valid2 = pd.read_csv('./chemberta2/results/clintox/chemberta2_valid2_clintox_3_predictions.csv')
ct_molformer_valid2 = pd.read_csv('./molformer/results/clintox/molformer_valid2_clintox_3_epoch26.csv')
ct_molbert_valid2 = pd.read_csv('./molbert/results/cilntox/molbert_valid2_clintox_3.csv')

# Load the test data for each model
ct_chemberta2_test = pd.read_csv('./chemberta2/results/clintox/chemberta2_test_clintox_3_predictions.csv')
ct_molformer_test = pd.read_csv('./molformer/results/clintox/molformer_test_clintox_3_epoch26.csv')
ct_molbert_test = pd.read_csv('./molbert/results/cilntox/molbert_test_clintox_3.csv')

# features

# Load the features from chemberta
ct_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/clintox/chemberta2_valid2_clintox_3_features.csv')
ct_chemberta2_features_test = pd.read_csv('./chemberta2/features/clintox/chemberta2_test_clintox_3_features.csv')

# Load the features from molformer
ct_molformer_features_valid2 = pd.read_csv('./molformer/features/clintox/molformer_valid2_clintox_3_features.csv')
ct_molformer_features_test = pd.read_csv('./molformer/features/clintox/molformer_test_clintox_3_features.csv')

# Load the features from molbert
ct_molbert_features_valid2 = pd.read_csv('./molbert/features/clintox/molbert_valid2_clintox_3_features.csv')
ct_molbert_features_test = pd.read_csv('./molbert/features/clintox/molbert_test_clintox_3_features.csv')


For Clintox (Classification)

In [13]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
ct_chemberta_actual = ct_chemberta2_test['CT_TOX']
ct_chemberta_pred = ct_chemberta2_test['y_pred']
ct_chemberta_probs = ct_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
ct_molformer_actual = ct_molformer_test['Actual']
ct_molformer_pred = (ct_molformer_test['Prob_Class_1'] > 0.5).astype(int)
ct_molformer_probs = ct_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
ct_molbert_actual = ct_molbert_test['target']
ct_molbert_pred = ct_molbert_test['pred']
ct_molbert_probs = ct_molbert_test['prob']

# Calculating metrics
ct_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", ct_chemberta_actual, ct_chemberta_pred, ct_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", ct_molformer_actual, ct_molformer_pred, ct_molformer_probs['Prob_Class_1']),
                                         ("Molbert", ct_molbert_actual, ct_molbert_pred, ct_molbert_probs)]:
    ct_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

ct_metrics_results

{'Chemberta2': {'Accuracy': 0.9731543624161074,
  'F1 Score': 0.8,
  'ROC-AUC': 0.9964028776978417,
  'PR-AUC': 0.9540404040404039},
 'Molformer': {'Accuracy': 0.9328859060402684,
  'F1 Score': 0.375,
  'ROC-AUC': 0.9266187050359712,
  'PR-AUC': 0.5340980864486556},
 'Molbert': {'Accuracy': 0.9172413793103448,
  'F1 Score': 0.14285714285714285,
  'ROC-AUC': 0.9133333333333333,
  'PR-AUC': 0.46665875863902184}}

In [14]:
# Determine the 'smiles' values in chemberta that are not in molbert
missing_smiles = set(ct_chemberta2_valid2['smiles']) - set(ct_molbert_valid2['smiles'])

# Find the indices of these missing 'smiles' in the chemberta DataFrame
missing_indices = ct_chemberta2_valid2.index[ct_chemberta2_valid2['smiles'].isin(missing_smiles)].tolist()

# Drop the rows with these missing indices for chemberta and molformer
ct_chemberta2_valid2 = ct_chemberta2_valid2.drop(missing_indices)
ct_molformer_valid2 = ct_molformer_valid2.drop(missing_indices)
ct_chemberta2_features_valid2 = ct_chemberta2_features_valid2.drop(missing_indices)
ct_molformer_features_valid2 = ct_molformer_features_valid2.drop(missing_indices)
ct_molbert_features_valid2 = ct_molbert_features_valid2.drop(missing_indices)

# do the same for test sets
missing_smiles = set(ct_chemberta2_test['smiles']) - set(ct_molbert_test['smiles'])
missing_indices = ct_chemberta2_test.index[ct_chemberta2_test['smiles'].isin(missing_smiles)].tolist()

ct_chemberta2_test = ct_chemberta2_test.drop(missing_indices)
ct_molformer_test = ct_molformer_test.drop(missing_indices)
ct_chemberta2_features_test = ct_chemberta2_features_test.drop(missing_indices)
ct_molformer_features_test = ct_molformer_features_test.drop(missing_indices)
ct_molbert_features_test = ct_molbert_features_test.drop(missing_indices)

In [15]:
# check shapes
print(ct_chemberta2_valid2.shape)
print(ct_molformer_valid2.shape)
print(ct_molbert_valid2.shape)
print(ct_chemberta2_features_valid2.shape)
print(ct_molformer_features_valid2.shape)
print(ct_molbert_features_valid2.shape)

(295, 8)
(295, 5)
(295, 4)
(295, 386)
(295, 769)
(295, 769)


In [16]:
import pandas as pd
import numpy as np

ct_y_ensemble_valid2 = ct_chemberta2_valid2['CT_TOX']

# Convert the ensemble target to a Series if not already done
ct_y_ensemble_valid2_s = pd.Series(ct_y_ensemble_valid2).reset_index(drop=True)

# Create dataframes for each model's class 1 probability
ct_chemberta2_prob = pd.DataFrame({'chemberta2': ct_chemberta2_valid2['softmax_class_1_prob']})
ct_chemberta2_prob.reset_index(drop=True, inplace=True)

ct_molformer_prob = pd.DataFrame({'molformer': ct_molformer_valid2['Prob_Class_1']})
ct_molformer_prob.reset_index(drop=True, inplace=True)

ct_molbert_prob = pd.DataFrame({'molbert': ct_molbert_valid2['prob']})
ct_molbert_prob.reset_index(drop=True, inplace=True)

# do the same for features ct_chemberta2_features_valid2.iloc[:, 2:]
ct_chemberta2_features = pd.DataFrame(ct_chemberta2_features_valid2.iloc[:, 2:])
ct_chemberta2_features.reset_index(drop=True, inplace=True)

ct_molformer_features = pd.DataFrame(ct_molformer_features_valid2.iloc[:, 1:])
ct_molformer_features.reset_index(drop=True, inplace=True)

ct_molbert_features = pd.DataFrame(ct_molbert_features_valid2.iloc[:, 1:])
ct_molbert_features.reset_index(drop=True, inplace=True)

# ct_features = pd.concat([ct_chemberta2_features, ct_molformer_features, ct_molbert_features], axis=1)

# Combine probabilities into one dataframe
train_ct_prob = pd.concat([ct_chemberta2_prob, ct_molformer_prob, ct_molbert_prob], axis=1)

# Function to calculate BCE for each row
def calculate_bce_rowwise(y_true, y_pred):
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Calculate row-wise BCE for each model
bce_chemberta = calculate_bce_rowwise(ct_y_ensemble_valid2_s, ct_chemberta2_prob['chemberta2'])
bce_molformer = calculate_bce_rowwise(ct_y_ensemble_valid2_s, ct_molformer_prob['molformer'])
bce_molbert = calculate_bce_rowwise(ct_y_ensemble_valid2_s, ct_molbert_prob['molbert'])

# Create a dataframe for row-wise BCE losses
bce_loss_df = pd.DataFrame({
    'bce_chemberta': bce_chemberta,
    'bce_molformer': bce_molformer,
    'bce_molbert': bce_molbert
})

# Final ensemble X matrix: Combine row-wise BCE losses, predictions, and features
ct_X_ensemble_valid2 = pd.concat([bce_loss_df, train_ct_prob], axis=1)

In [17]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Combine probabilities with their respective feature sets
chemberta_X = pd.concat([ct_chemberta2_prob, ct_chemberta2_features], axis=1)
molformer_X = pd.concat([ct_molformer_prob, ct_molformer_features], axis=1)
molbert_X = pd.concat([ct_molbert_prob, ct_molbert_features], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Define the binary cross-entropy loss values as target variables (y)
chemberta_y_bce = bce_chemberta  # Row-wise BCE loss calculated earlier
molformer_y_bce = bce_molformer  # Row-wise BCE loss calculated earlier
molbert_y_bce = bce_molbert      # Row-wise BCE loss calculated earlier

# Initialize the SVR models
svr_chemberta = SVR()
svr_molformer = SVR()
svr_molbert = SVR()

# Setup cross-validation for parameter tuning using neg_mean_squared_error as the metric
param_grid = {'C': np.logspace(-3, 2, 6), 'epsilon': [0.1, 0.2, 0.5], 'kernel': ['linear', 'rbf']}

cv_chemberta = GridSearchCV(estimator=svr_chemberta, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molformer = GridSearchCV(estimator=svr_molformer, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
cv_molbert = GridSearchCV(estimator=svr_molbert, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit GridSearchCV for each dataset with BCE as the target
cv_chemberta.fit(chemberta_X_scaled, chemberta_y_bce)
cv_molformer.fit(molformer_X_scaled, molformer_y_bce)
cv_molbert.fit(molbert_X_scaled, molbert_y_bce)

# Retrieve the best models and parameters
best_model_chemberta = cv_chemberta.best_estimator_
best_model_molformer = cv_molformer.best_estimator_
best_model_molbert = cv_molbert.best_estimator_

In [18]:
import numpy as np
from sklearn.metrics import log_loss

# Test data for each model
ct_chemberta2_prob_test = pd.DataFrame({'chemberta2': ct_chemberta2_test['softmax_class_1_prob']})
ct_chemberta2_prob_test.reset_index(drop=True, inplace=True)

ct_molformer_prob_test = pd.DataFrame({'molformer': ct_molformer_test['Prob_Class_1']})
ct_molformer_prob_test.reset_index(drop=True, inplace=True)

ct_molbert_prob_test = pd.DataFrame({'molbert': ct_molbert_test['prob']})
ct_molbert_prob_test.reset_index(drop=True, inplace=True)

ct_chemberta2_features_t = pd.DataFrame(ct_chemberta2_features_test.iloc[:, 2:])
ct_chemberta2_features_t.reset_index(drop=True, inplace=True)

ct_molformer_features_t  = pd.DataFrame(ct_molformer_features_test.iloc[:, 1:])
ct_molformer_features_t.reset_index(drop=True, inplace=True)

ct_molbert_features_t = pd.DataFrame(ct_molbert_features_test.iloc[:, 1:])
ct_molbert_features_t.reset_index(drop=True, inplace=True)

# Combine probabilities with the respective feature sets for the test set
chemberta_X_test = pd.concat([ct_chemberta2_prob_test, ct_chemberta2_features_t], axis=1)
molformer_X_test = pd.concat([ct_molformer_prob_test, ct_molformer_features_t], axis=1)
molbert_X_test = pd.concat([ct_molbert_prob_test, ct_molbert_features_t], axis=1)

# Standardize the test set based on the previously fitted scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Predict using the best models from valid2
chemberta_pred_test = best_model_chemberta.predict(chemberta_X_test_scaled)
molformer_pred_test = best_model_molformer.predict(molformer_X_test_scaled)
molbert_pred_test = best_model_molbert.predict(molbert_X_test_scaled)

# Convert the predictions (numpy arrays) to pandas Series
chemberta_pred_test_series = pd.Series(chemberta_pred_test, name='bce_chemberta')
molformer_pred_test_series = pd.Series(molformer_pred_test, name='bce_molformer')
molbert_pred_test_series = pd.Series(molbert_pred_test, name='bce_molbert')

# Now concatenate the series with the test set probabilities
ct_X_ensemble_test = pd.concat([
    chemberta_pred_test_series,                     # BCE for Chemberta
    molformer_pred_test_series,                     # BCE for Molformer
    molbert_pred_test_series,                       # BCE for Molbert
    ct_chemberta2_prob_test['chemberta2'],        # Chemberta test probabilities
    ct_molformer_prob_test['molformer'],          # Molformer test probabilities
    ct_molbert_prob_test['molbert']               # Molbert test probabilities
], axis=1)

ct_X_ensemble_test.columns = ['bce_chemberta', 'bce_molformer', 'bce_molbert', 'chemberta2', 'molformer', 'molbert']

# optional for evaluation
ct_y_ensemble_test = ct_chemberta2_test['CT_TOX']

In [19]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
ct_X_ensemble_valid2_scaled = scaler.fit_transform(ct_X_ensemble_valid2)
ct_X_ensemble_test_scaled = scaler.transform(ct_X_ensemble_test)

# transform back to dataframe
ct_X_ensemble_valid2_scaled = pd.DataFrame(ct_X_ensemble_valid2_scaled, columns=ct_X_ensemble_valid2.columns)
ct_X_ensemble_test_scaled = pd.DataFrame(ct_X_ensemble_test_scaled, columns=ct_X_ensemble_test.columns)


In [20]:
ct_X_ensemble_valid2_selected = ct_X_ensemble_valid2_scaled
ct_X_ensemble_test_selected = ct_X_ensemble_test_scaled
# check shapes
print(ct_X_ensemble_valid2_selected.shape)
print(ct_X_ensemble_test_selected.shape)

(295, 6)
(145, 6)


In [21]:
# Initialize and train the SVM model
from sklearn.svm import SVC

ct_svm_model = SVC(probability=True)
ct_svm_model.fit(ct_X_ensemble_valid2_selected, ct_y_ensemble_valid2)

# Predict the test set
ct_svm_pred = ct_svm_model.predict(ct_X_ensemble_test_selected)
ct_svm_probs = ct_svm_model.predict_proba(ct_X_ensemble_test_selected)

# Calculate the metrics
ct_svm_metrics = {
    'Accuracy': accuracy_score(ct_y_ensemble_test, ct_svm_pred),
    'F1 Score': f1_score(ct_y_ensemble_test, ct_svm_pred),
    'ROC-AUC': roc_auc_score(ct_y_ensemble_test, ct_svm_probs[:, 1]),
    "PR-AUC": average_precision_score(ct_y_ensemble_test, ct_svm_probs[:, 1])
}

ct_svm_metrics

{'Accuracy': 0.9793103448275862,
 'F1 Score': 0.8421052631578947,
 'ROC-AUC': 0.997037037037037,
 'PR-AUC': 0.9651515151515151}

In [22]:
# report all the metrics for ct
ct_metrics_results["SVM"] = ct_svm_metrics

ct_metrics_df = pd.DataFrame(ct_metrics_results).T

# keep 3 digits after the decimal point
ct_metrics_df = ct_metrics_df.round(3)

# export as csv
ct_metrics_df.to_csv('./split3_ct_metrics_svm.csv')